In [ ]:
!pip install patool

In [ ]:
!pip install python-chess

In [ ]:
import chess
import numpy as np
import pandas as pd
import patoolib

from sklearn.model_selection import train_test_split

import tensorflow as tf

In [ ]:
example_fen = 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1'

In [ ]:
boardExample = chess.Board(example_fen)

In [ ]:
def bitboard(fen):
  board = chess.Board(fen)
  pieces = [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN, chess.KING]
  colors = [chess.WHITE, chess.BLACK]
  result = []
  for color in colors:
    for piece in pieces:
      pieceArray = board.pieces(piece, color)
      pieceList = np.array(pieceArray.tolist())
      pieceList = np.reshape(pieceList, (8, 8)).astype(int)
      result.append(pieceList)
  result = np.array(result)
  rotated = np.transpose(result, axes=(1, 2, 0))
  return rotated

In [ ]:
data = pd.read_csv("chessData.csv")

In [ ]:
X = []
Y = []
amount = 100 # rows of data
for index in data.index[:amount]:
  row = data.iloc[index]
  fen = row['FEN']
  eval = row['Evaluation']
  bitmap = bitboard(fen)
  if '#' in eval:
    eval = eval[1:]
  eval = int(eval)
  X.append(bitmap)
  Y.append(eval)
X = np.array(X)
Y = np.array(Y)

In [ ]:
X.shape

(100, 8, 8, 12)

In [ ]:
# Neural network part
i = tf.keras.layers.Input((8, 8, 12))
x1 = tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu')(i)
x2 = tf.keras.layers.Conv2D(8, (3, 3), activation = 'relu')(x1)
x3 = tf.keras.layers.Conv2D(4, (3, 3), activation = 'relu')(x2)
f1 = tf.keras.layers.Flatten()(x3)
d1 = tf.keras.layers.Dense(32, activation = 'relu')(f1)
d2 = tf.keras.layers.Dense(16, activation = 'relu')(d1)
d3 = tf.keras.layers.Dense(1)(d2) # This is the final value (evaluation) that the model will output.

model = tf.keras.models.Model(inputs=i, outputs = d3)







In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 8, 8, 12)]        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 6, 6, 16)          1744      
                                                                 
 conv2d_10 (Conv2D)          (None, 4, 4, 8)           1160      
                                                                 
 conv2d_11 (Conv2D)          (None, 2, 2, 4)           292       
                                                                 
 flatten_3 (Flatten)         (None, 16)                0         
                                                                 
 dense_9 (Dense)             (None, 32)                544       
                                                                 
 dense_10 (Dense)            (None, 16)                528   

In [ ]:
model.compile(loss = 'mse',
              optimizer = 'adam',
              metrics = ['mse'])

In [ ]:
# Training the model
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2) # 20% data for testing, and 80% data for training.

In [ ]:
model.fit(x_train, y_train, epochs = 500, batch_size = 4) # epochs is basically how many times it iterates through your data. batch_s

In [ ]:
def findMove(fen, turn='white'):
  board = chess.Board(fen)
  turn = 'white'
  if turn == 'white':
    bestMove = ''
    bestEval = -1000000
    newBoard = ''
    for move in board.legal_moves:
      boardCopy = board.copy()
      boardCopy.push(move)
      bitmap = bitboard(boardCopy.fen())
      bitmap = np.reshape(bitmap, (1, 8, 8, 12))
      eval = model.predict(bitmap, verbose=False)
      if eval > bestEval:
        bestEval = eval
        bestMove = move
        newBoard = boardCopy
  else:
    bestMove = ''
    bestEval = 1000000
    newBoard = ''
    for move in board.legal_moves:
      boardCopy = board.copy()
      boardCopy.push(move)
      bitmap = bitboard(boardCopy.fen())
      bitmap = np.reshape(bitmap, (1, 8, 8, 12))
      eval = model.predict(bitmap, verbose=False)
      if eval < bestEval:
        bestEval = eval
        bestMove = move
        newBoard = boardCopy
  return bestMove

In [ ]:
startBoard = chess.Board('rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 1')
print(startBoard)
userinput = input('Enter your first move: ')
while userinput != 'end':
  move = chess.Move.from_uci(userinput)
  startBoard.push(move)
  botMove = findMove(startBoard.fen(), turn = 'black')
  startBoard.push(botMove)
  print()
  print()
  print(startBoard)
  userinput = input('Enter your next move: ')

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . P . . .
. . N . . . . .
P P P P . P P P
R . B Q K B N R


r . b q k b n r
p p p n p p p p
. . . . . . . .
. . . P . . . .
. . . . . . . .
. . N . . . . .
P P P P . P P P
R . B Q K B N R


r . b q k b n r
p p p n . p p p
. . . . p . . .
. . . P . . . .
. . . . . . . .
. . N P . . . .
P P P . . P P P
R . B Q K B N R


r . b q k b n r
p p p n . . p p
. . . . p . . .
. . . . . . . .
. . . . . . . .
. . N P . . . .
P P P . . P P P
R . B Q K B N R


r . b Q . b n r
p p p n . k p p
. . . . p . . .
. . . . . . . .
. . . . . . . .
. . N P . . . .
P P P . . P P P
R . B . K B N R


r . b . . b . r
p p p n . Q p p
. . . . p n . .
. . . . . . . .
. . . . . . . .
. . N P . . . .
P P P . . P P P
R . B . K B N R
